# CLWW vs Ideal ORE

In [59]:
import collections
import math
from math import floor, log2
import numpy as np
import pandas as pd
from scipy.special import perm # for nPk 
from sympy.functions.combinatorial.numbers import stirling
from decimal import Decimal
import matplotlib.pyplot as plt 
pd.set_option('precision', 10)
%matplotlib inline

## r(i, k) table

In [3]:
#f[n] = f[n-1] + f[n-2]
def fib(n):
    s = [1, 1]
    if n == 1:
        return [1]
    elif n == 2: 
        return [1, 1]
    else:
        for i in range(n):
            s.append(s[-1] + s[-2])
        return s

In [4]:
print(fib(10))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]


In [35]:
def print_array(a):
    k = 0
    for l in a:
        print("k: ", k , l)
        k = k + 1

In [53]:
def trailing_zero(n):
    x = int(bin(n)[2:])
    count = 0
    while ((x & 1 ) == 0):
        x = x >> 1
        count += 1
    return count

In [54]:
bt = bin(10)
print(bt)
print(trailing_zero(10))

0b1010
1


In [61]:
def report_number(I, K):
    r = [[0]* (I + 1) for _ in range(K+1)]
    r[1][1] = 1
    for k in range(2, K):
        b = trailing_zero(k-1)
        for i in range(1, I):
            r[i][k] = r[i][k-1] + r[i-1][k-1] - r[i-1][k-1- int(math.pow(2,b))]
    return r

In [62]:
i = 32
k = 32
a = report_number(i, k)
print_array(a)

k:  0 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
k:  1 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
k:  2 [0, 0, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0]
k:  3 [0, 0, 0, 1, 2, 4, 5, 6, 6, 9, 10, 11, 11, 12, 12, 12, 12, 16, 17, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 0]
k:  4 [0, 0, 0, 0, 1, 3, 5, 8, 9, 15, 18, 22, 23, 28, 29, 30, 30, 42, 46, 51, 52, 58, 59, 60, 60, 67, 68, 69, 69, 70, 70, 70, 0]
k:  5 [0, 0, 0, 0, 0, 1, 3, 7, 10, 19, 25, 34, 38, 52, 57, 63, 64, 94, 106, 122, 127, 149, 155, 162, 163, 193, 200, 208, 209, 218, 219, 220, 0]
k:  6 [0, 0, 0, 0, 0, 0, 1, 4, 8, 18, 27, 42, 51, 79, 93, 112, 118, 182, 212, 254, 270, 333, 355, 383, 390, 489, 519, 556, 564, 610, 619, 629, 0]
k:  7 [0, 0, 0, 0, 0, 0, 0, 1, 4, 12, 22, 41, 56, 99, 127, 169, 188, 306, 370, 464, 506, 658, 721, 806, 834, 1106, 1205, 1334, 1371, 15

## CLWW PBV

__uni_ore_clww_bayes(n, k)__ calculates the posterior Bayes vulnerability for CLWW Order Revealing Encryption (ORE). Only works on a uniform prior

$$ V_1 [C \triangleright \pi ]  =  \frac{\sum_{i=1}^{min(n, k)} \text{stirling2}(n, i)\times i! \times \text{r}(i, k)}{\sum_{i=1}^{min(n, k)} \text{stirling2}(n, i)\times kPi}  $$

The numerator is larger than ideal ORE (more channel outputs)

The denominator is the same as ideal ORE (same number of channel inputs)

In [ ]:
def uni_ore_clww_bayes(n, k, M): # M is df matrix, 1 row
    outputs = 0
    inputs = 0
    for i in range(n):
        if k < i+1:
            break
        s2 = stirling(n, i+1)
        # need to check the indexing on new M
        outputs = outputs + (math.factorial(i+1) * s2 * M.iloc[0, i+1])
        inputs = inputs + (perm(k, i+1) * s2)
    return outputs / inputs

## IDEAL PBV

__uni_ore_bayes__ calculates the posterior Bayes vulnerability for ideal Order Revealing Encryption (ORE). Only works on a uniform prior for now. 

Number of channel outputs over number of channel inputs

$$ V_1 [C \triangleright \pi ]  =  \frac{\sum_{i=1}^{min(n, k)} \text{stirling2}(n, i)\times i!}{\sum_{i=1}^{min(n, k)} \text{stirling2}(n, i)\times kPi}  $$


outputs:= channel outputs (ways to order integer partitions), the Fubini numbers (ordered Bell numbers) a sum of the stirling number of the second kind (n, i) times i!

inputs := channel inputs, a sum of Stirling numbers of the second kind (n, i) times kPi. (equals k^n)
This total number of channel rows can be calculated by k^n but this triggers an overflow in Python.

In [ ]:
def uni_ore_bayes(n, k):
    outputs = 0
    inputs = 0
    for c in range(n):
        if k < c+1:
            break
        s2 = stirling(n, c+1)
        outputs = outputs + (math.factorial(c+1) * s2)
        inputs = inputs + (perm(k, c+1) * s2)
    return outputs / inputs

## IDEAL VS CLWW

### General Graph Code

In [ ]:
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator
font_choice = 14
plt.rcParams.update(plt.rcParamsDefault)
rc_fonts = {
    "text.usetex": True,
    "font.size": font_choice,
    'mathtext.default': 'regular',
    'axes.titlesize': font_choice,
    "axes.labelsize": font_choice,
    "legend.fontsize": font_choice,
    "xtick.labelsize": font_choice,
    "ytick.labelsize": font_choice,
    'figure.titlesize': font_choice,
    'figure.figsize': (8,5.25),
    'text.latex.preamble': [r'\usepackage{amsmath,nicefrac, sansmath}', 
                            r'\sansmath'],
    "font.family": "sans-serif",#"font.sans-serif": "computer modern",
    }
mpl.rcParams.update(rc_fonts)

### Function of n

#### Function of n, df

In [ ]:
def df_uni_ore_idealvsCLWW_bayes(n_range, k, M): #M is df matrix
    n = 0
    n_list = []
    prior_list = []
    postIdeal_list = []
    postCLWW_list = []
    for x in range(n_range):
        n = n + 1
        n_list.append(n)
        prior = pow(1/k, n)
        prior_list.append(prior)
        clww = uni_ore_clww_bayes(n, k, M) #only dif
        postCLWW_list.append(clww)
        ideal = uni_ore_bayes(n, k)
        postIdeal_list.append(ideal)
    df = pd.DataFrame(
        {'n':n_list,
         'prior':prior_list,
         'postIdeal':postIdeal_list,
         'postCLWW': postCLWW_list
        }
    )
    return df

#### Function of n, graph code

In [ ]:
def graph_it_vs(n_range, k, df):
    ax = plt.figure().gca()
    plt.plot('n', 'prior', 
             data=df,
             label="prior")
    plt.plot('n', 'postIdeal',
             color='green',
             data=df,
             linestyle='--',
             marker='', 
             label="Ideal")
    plt.plot('n', 'postCLWW',
             color='red',
             data=df,
             linestyle='dotted',
             marker='', 
             label="CLWW")
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.xlabel('n')
    plt.xlim(left = 0, right = n_range+1)
    plt.ylabel('Bayes vulnerability')
    plt.ylim(bottom=-.1, top = 1)
    
    plt.title('CLWW vs Ideal ORE Bayes vulnerability, k = {}'.format(k))
    plt.legend(loc='upper left') # legend adjusted so it doesn't run into annotations
    plt.show()
    plt.close()

#### Function of n, graph

In [ ]:
n_range =
k = 
graph_it_vs(n_range, k, M)

### Function of k

#### Function of k, df

#### Function of k, graph code

#### Function of k, graphs

### N and K together

#### N and K together, df

#### N and K together, graph code

#### N and K together, graphs

### k = 2n

#### k = 2n, df

#### k = 2n, graph code

#### k = 2n, graphs